## Prerequisites



In [0]:
import pandas as pd 
import numpy as np 
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline

### Note! Some of these models support only multiclass classification, please, while selecting your dataset,  
### be sure that for algorithms which does not support multilabel classification you use only examples with only one label. 
### Examples without a label in any of the provided categories are clean messages, without any toxicity.

In [0]:
df = pd.read_csv("train.csv")

In [0]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [0]:
df.shape

(159571, 8)

### As one of the methods to make the training simpier, use only examples, assigned to any category vs clean examples.  
For example:  
- Select only messages with obscene label == 1  
- Select all of the "clean" messages  
Implement a model which can perform a binary classification  - to understand whether your message is obscene or not.   

##### If you want to perform a multilabel classification, please understand the difference between multilabel and multiclass classification and be sure that you are solving the correct task - choose only algorithms applicable for solving this type of problem.

#### To work with multiclass task:  
You only need to select messages which have only one label assigned: message cannot be assigned to 2 or more categories.  

#### To work with multilabel task: 
You can work with the whole dataset - some of your messages have only 1 label, some more than 1. 

## Text vectorization

Previously we worked only with words vectorization. But we need to have a vector for each text, not only words from it. 

Before starting a text vectorization, please, make sure you are working with clean data - use the dataset created on the previous day. Cleaned from punctuation, stop words, lemmatized or stemmed, etc. 

In [34]:
from string import punctuation

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('stopwords')  
lemmatizer = WordNetLemmatizer() 
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
def preprocess_text(tokenizer, lemmatizer, stop_words, punctuation, text): 
    tokens = tokenizer(text.lower())
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    return [token for token in lemmas if token not in stop_words and token not in punctuation]

df['cleaned'] = df.comment_text.apply(lambda x: preprocess_text(word_tokenize, lemmatizer, stop_words, punctuation, x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [36]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,cleaned
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0,"[explanation, edits, made, username, hardcore,..."
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[d'aww, match, background, colour, 'm, seeming..."
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[hey, man, 'm, really, trying, edit, war, 's, ..."
3,0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0,"[``, ca, n't, make, real, suggestion, improvem..."
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[sir, hero, chance, remember, page, 's]"


In [0]:
def flat_nested(nested):
    flatten = []
    for item in nested:
        if isinstance(item, list):
            flatten.extend(item)
        else:
            flatten.append(item)
    return flatten

In [0]:
vocab = set(flat_nested(df.cleaned.tolist()))

In [0]:
len(vocab)

249736

As we see, probably you vocabulary is too large.  
Let's try to make it smaller.  
For example, let's get rig of words, which has counts in our dataset less than some threshold.

In [0]:
from collections import Counter, defaultdict 

cnt_vocab = Counter(flat_nested(df.cleaned.tolist()))

In [64]:
cnt_vocab.most_common(10)

[("''", 242528),
 ('``', 155370),
 ('article', 73284),
 ("'s", 66767),
 ("n't", 57144),
 ('wa', 56592),
 ('page', 56263),
 ('wikipedia', 45418),
 ('talk', 35356),
 ('ha', 31896)]

You can clean words which are shorter that particular length and occur less than N times. 

In [0]:
threshold_count = 10
threshold_len = 4 
cleaned_vocab = [token for token, count in cnt_vocab.items() if count > threshold_count and len(token) > threshold_len]

In [9]:
len(cleaned_vocab)

18696

Much better!  
Let's try to vectorize the text summing one-hot vectors for each word. 

In [0]:
vocabulary = defaultdict()

for i, token in enumerate(cleaned_vocab): 
    empty_vec = np.zeros(len(cleaned_vocab))
    empty_vec[i] = 1 
    vocabulary[token] = empty_vec


In [83]:
vocabulary['source']

array([0., 0., 0., ..., 0., 0., 0.])

Rigth now we have vectors for words (words are one-hot 
vectorized**)  
Let's try to create vectors for texts: 

In [42]:
sample_text = df.cleaned[10]
print(sample_text)

['``', 'fair', 'use', 'rationale', 'image', 'wonju.jpg', 'thanks', 'uploading', 'image', 'wonju.jpg', 'notice', 'image', 'page', 'specifies', 'image', 'used', 'fair', 'use', 'explanation', 'rationale', 'use', 'wikipedia', 'article', 'constitutes', 'fair', 'use', 'addition', 'boilerplate', 'fair', 'use', 'template', 'must', 'also', 'write', 'image', 'description', 'page', 'specific', 'explanation', 'rationale', 'using', 'image', 'article', 'consistent', 'fair', 'use', 'please', 'go', 'image', 'description', 'page', 'edit', 'include', 'fair', 'use', 'rationale', 'uploaded', 'fair', 'use', 'medium', 'consider', 'checking', 'specified', 'fair', 'use', 'rationale', 'page', 'find', 'list', "'image", 'page', 'edited', 'clicking', '``', "''", 'contribution', "''", "''", 'link', 'located', 'top', 'wikipedia', 'page', 'logged', 'selecting', '``', "''", 'image', "''", "''", 'dropdown', 'box', 'note', 'fair', 'use', 'image', 'uploaded', '4', 'may', '2006', 'lacking', 'explanation', 'deleted', 'one

### One-hot vectorization and count vectorization

In [0]:
sample_vector = np.zeros(len(cleaned_vocab))

for token in sample_text: 
    try: 
        sample_vector += vocabulary[token]
    except KeyError: 
        continue   

In [0]:
sample_vector

array([3., 0., 0., ..., 0., 0., 0.])

Right now we have count vectorization for our text.   
Use this pipeline to create vectors for all of the texts. Save them into np.array. i-th raw in np.array is a vector which represents i-th text from the dataframe.  

In [0]:
sample_size=1000
df_small = df.cleaned.sample(sample_size)
def vectorization(shape, data):
   k=0
   all_text_vector = np.zeros((shape, len(cleaned_vocab)))
   for i in data:
     text_vector = np.zeros(len(cleaned_vocab))
     for token in i: 
       try: 
           text_vector += vocabulary[token]
       except KeyError: 
           continue 
     all_text_vector[k]=text_vector
     k+=1
   return  all_text_vector
vectorize_text = vectorization (sample_size, df_small)


In [139]:
vectorize_text

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [45]:
idx = df_small.index
texts = df.cleaned[idx]
test_vectorize = vectorization(sample_size, texts)
test_arr = test_vectorize == vectorize_text
test=np.all(test_arr)
print(test)


True


### The next step is to train any classification model on top of the received vectors and report the quality. 

Please, select any of the proposed pipelines for performing a text classification task. (Binary, multiclass or multilabel).  

The main task to calculate our models performance is to create a training and test sets. When you selected a texts for your task, please, use https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html to have at least two sets - train and test.  

Train examples you will use to train your model on and test examples to evaluate your model - to understand how your model works on the unseen data. 

### Train-test split 

In [46]:
p=0.75
df_categories = ['identity_hate', 'insult', 'obscene', 'severe_toxic', 'threat', 'toxic']
df_insult = df[df['insult']==1]
df_clean = df[~df[df_categories].any(axis = 'columns')]
df_combined = df_insult.append(df_clean)
df_combined = df_combined[['cleaned', 'insult']]
df_combined = df_combined.sample(frac = 1)
df_train, df_test = train_test_split(df_combined, train_size = p)
X_train = df_train['cleaned'] 
Y_train = df_train['insult']
X_test = df_test['cleaned']
Y_test = df_test['insult']
X_train

129305    [``, yes, n't, understand, reason, ``, '', int...
155946                         [put, another, one, bracket]
70682     [``, sorry, delay, 've, semi-protected, articl...
154634    [georges-henri, pingusson, hi, thanks, ref, edit]
78586              [sure, anything, improve, article, talk]
                                ...                        
158048    [``, think, put, needed, talk, page, dpeterson...
111148    [``, ha, since, resurrected, see, ref, article...
121889    [distinction, would, draw, whether, content, p...
134107    [norma, writing, end, every, edit, 's, descrip...
153320    [``, 'm, confused, statement, ``, '', 95, free...
Name: cleaned, Length: 113417, dtype: object

TF-IDF score 

#### Please, review again this article or read it if you have not done it before. 

https://medium.com/@paritosh_30025/natural-language-processing-text-data-vectorization-af2520529cf7

#### Implement calculating a tf-idf score for each of the words from your vocabulary. 

The main goal of this taks is to create a dictionary - keys of the dictionary would be tokens and values would be corresponding tf-idf score of the token.

#### Calculate it MANUALLY and compare the received scores for words with the sklearn implementation:  
from sklearn.feature_extraction.text import TfidfTransformer 

#### Tip: 

##### TF = (Number of time the word occurs in the current text) / (Total number of words in the current text)  

##### IDF = (Total number of documents / Number of documents with word t in it)

##### TF-IDF = TF*IDF 

When you calculated a tf-idf score for each of the words in your vocabulary - revectorize the texts.  
Instead of using number of occurences of the i-th word in the i-th cell of the text vector, use it's tf-idf score.   

Revectorize the documents, save vectors into np.array. 

In [47]:
vectorizer = CountVectorizer()
X_train_list=X_train.to_list()
corpus=[]
for text in X_train_list:
  corpus.append(' '.join(text))
word_count_vector = vectorizer.fit_transform(corpus)
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=vectorizer.get_feature_names(),columns=["idf_weights"])
count_vector=vectorizer.transform(corpus)
tf_idf_vector=tfidf_transformer.transform(count_vector)
feature_names = vectorizer.get_feature_names()
first_document_vector=tf_idf_vector[0]
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
df_dict = {x: float(y) for x, y in zip(df.sort_values(by=["tfidf"],ascending=False).index[:10], df.sort_values(by=["tfidf"],ascending=False).values[:10])}
print(df_dict)

{'interrupted': 0.6522249007340178, 'deleed': 0.40992295047881866, 'comprende': 0.3960091892498387, 'template': 0.34101932114933237, 'wo': 0.18511897659570867, 'yes': 0.1631708936040326, 'understand': 0.15810721733768995, 'reason': 0.14207373720962407, 'edits': 0.1391981905874131, 'wa': 0.08941838126809708}


### Training the model 

As it was said before, select any of the text classification models for the selected task and train the model. 

When the model is trained, you need to evaluate it somehow. 

Read about True positive, False positive, False negative and True negative counts and how to calculate them:   

https://developers.google.com/machine-learning/crash-course/classification/true-false-positive-negative 

##### Calculate TP, FP, FN and TN on the test set for your model to measure its performance. 


In [0]:
X_train_pre = X_train.str.join(' ')
X_test_pre = X_test.str.join(' ')
pipeline = Pipeline([('matrix', CountVectorizer(vocabulary = cleaned_vocab)),('tf-idf', TfidfTransformer())])
X_train_tf_idf = pipeline.fit_transform(X_train_pre)
X_test_tf_idf = pipeline.transform(X_test_pre)

In [49]:
linear_classifier = LogisticRegression(random_state = 0).fit(X_train_tf_idf, Y_train)

Y_pred = linear_classifier.predict(X_test_tf_idf)

Y_pred.shape==Y_test.shape

True

In [50]:
TN =  np.sum([(Y_test==Y_pred) & Y_test]) 
TP =  np.sum([(Y_test==Y_pred) & ~Y_test])
FN =  np.sum([(Y_test!=Y_pred) & Y_test])
FP =  np.sum([~((Y_test==Y_pred) | Y_test)])
    
  

print(' TN=', TN, '\n', 'TP=', TP, '\n', 'FN=', FN, '\n',  'FP=', FP)


 TN= 952 
 TP= 35818 
 FN= 985 
 FP= 51


#### The next step is to calculate  Precision, Recall, F1 and F2 score 

https://en.wikipedia.org/wiki/Sensitivity_and_specificity

In [51]:
transpose_matr = np.transpose(np.array([[TP, FP], [FN, TN]]))
PRE = transpose_matr[0,0]/np.sum(transpose_matr[0,:])
REC = transpose_matr[0,0]/np.sum(transpose_matr[:,0])
F1 = (1 + 1) * (PRE * REC) / (PRE + REC)
F2 = (1 + 2 ** 2) * (PRE * REC) / (((2 ** 2) * PRE) + REC)
print(' PRE=', PRE, '\n', 'REC=', REC, '\n', 'F1=', F1, '\n', 'F2=', F2 )
    
    



 PRE= 0.9732358775099856 
 REC= 0.9985781594134211 
 F1= 0.98574416556583 
 F2= 0.9934046672102684


Calculate these metrics for the vectorization created using count vectorizing and for tf-idf vectorization.  
Compare them. 

In [55]:
df_train_1, df_test_1 = train_test_split(df.sample(sample_size), train_size_1 = p)
Y_train_1 = df_train_1['insult']
linear_classifier = LogisticRegression(random_state = 0).fit(vectorize_text, Y_train_1)

Y_pred = linear_classifier.predict(X_test_tf_idf)

Y_pred.shape==Y_test.shape

TypeError: ignored

### Conclusions and improvements 

For all of the vectorization pipelines we used all of the words, which were available in our dictionary, as experiment try to use the most meaningful words - select them using TF-IDF score. (for example for each text you can select not more than 10 words for vectorization, or less). 

Compare this approach with the first and second ones. Did your model improve? 



### Additionally, visualisations 

For now you have a vector for each word from your vocabulary. 
You have vectors with lenght > 18000, so the dimension of your space is more than 18000 - it's impossible to visualise it in 2d space. 

So try to research and look for algorithms which perform dimensionality reduction. (t-SNE, PCA) 
Try to visualise obtained vectors in a vectorspace, only subset from the vocabulary, don't plot all of the words. (100) 

Probably on this step you will realise how this type of vectorization using these techniques is not the best way to vectorize words. 

Please, analyse the obtained results and explain why visualisation looks like this. 